In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
dataset_path = 'Twitter_Data.csv'
df = pd.read_csv(dataset_path)
df

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
57848,and clarify you ardent karyakarta for trs when...,0.0
57849,during before announced feat scientists today ...,1.0
57850,why would act helps modi,0.0
57851,are with modi and modi with and you are useles...,-1.0


## Xóa missing data:
Bộ dữ liệu này có tồn tại một vài hàng chứa giá trị null:

In [ ]:
df = df.dropna()

## Tiền xử lý bộ dữ liệu:
Dữ liệu đầu vào của chúng ta lúc này hiện đang ở dạng văn bản (string), chưa có đặc trưng rõ ràng cũng như không thể đưa vào huấn luyện mô hình được. Vì vậy, chúng ta sẽ tiền xử lý dữ liệu văn bản đầu vào để đưa về một dạng
vector đặc trưng nào đó:

### (a) Xây dựng hàm chuẩn hóa văn bản:
Văn bản gốc có rất nhiều kí tự dư thừa, vô nghĩa... Vì vậy, ta cần loại bỏ chúng cũng như áp dụng thêm vài bước chuẩn hóa văn bản khác để văn bản đầu vào trở nên ít phức tạp hơn, nhằm tăng cường hiệu quả biểu diễn của vector đặc trưng sau này:

In [ ]:
def text_normalize(text):
    # Lowercasing
    text = text.lower()

    # Retweet old acronym "RT" removal
    text = re.sub(r'^rt[\s]+', '', text)

    # Hyperlinks removal
    text = re.sub(r'https?:\/\/.*[\ r\n]*', '', text)

    # Punctuation removal
    text = re.sub(r'[^\ w\s]', '', text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))

    words = text.split()
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)

    # Stemming
    stemmer = SnowballStemmer ('english')
    words = text.split()
    words = [stemmer.stem(word) for word in words]
    text = ' '.join(words)

    return text

### (b) Khởi tạo tf-idf vectorizer:
Trong bài này, chúng ta sẽ sử dụng một dạng vector biểu diễn đặc trưng mới cho văn bản, đó là tf-idf. Trong phạm vi của bài, chúng ta sẽ không đi sâu vào kỹ thuật tf-idf, các bạn có thể đọc thêm về kỹ thuật này tại [đây](https://www.geeksforgeeks.org/understanding-tf-idf-term-frequency-inverse-document-frequency/). Code biến đổi văn bản đầu vào thành vector tf-idf như sau:

In [ ]:
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(df['clean_text']).toarray()

### (c) Thêm bias vào X:

In [ ]:
intercept = np.ones((X.shape[0], 1))
X_b = np.concatenate((intercept,X), axis=1)

## One-hot encoding label:

In [ ]:
n_classes = df['category'].nunique()
n_samples = df['category'].size

y = df['category'].to_numpy() + 1
y = y.astype(np.uint8)
y_encoded = np.array([np.zeros(n_classes) for _ in range(n_samples)])
y_encoded[np.arange(n_samples), y] = 1

## Chia bộ train, val, test:

In [ ]:
val_size = 0.2
test_size = 0.125
random_state = 2
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(X_b, y_encoded,
                                                  test_size=val_size,
                                                  random_state=random_state,
                                                  shuffle=is_shuffle)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
                                                    test_size = test_size,
                                                    random_state = random_state,
                                                    shuffle = is_shuffle)

## Cài đặt các hàm quan trọng:

In [10]:
def softmax(z):
  exp_z = np.exp(z)
  return exp_z / exp_z.sum(axis=1)[:, None]

def predict(X, theta):
  z = np.dot(X, theta)
  y_hat = softmax(z)
  return y_hat

def compute_loss(y_hat, y):
  n = y.size

  return round((-1 / n) * np.sum(y * np.log(y_hat)), 8)

def compute_gradient(X, y, y_hat):
  n = y.size

  return np.dot(X.T, (y_hat - y)) / n

def update_theta(theta, gradient, lr):
  return theta - lr * gradient

def compute_accuracy(X, y, theta):
  y_hat = predict(X, theta)
  acc = (np.argmax(y_hat, axis=1) == np.argmax(y, axis=1)).mean()
  return acc

## Khai báo các siêu tham số và khởi tạo weights: Trong bài này, vì số lượng mẫu
dữ liệu là rất lớn, ta có thể cân nhắc tăng số batch size lên để tăng tốc độ huấn luyện (ví dụ ở đây ta cài batch_size=n_samples).

In [11]:
lr = 0.1
epochs = 200
batch_size = X_train.shape[0]
n_features = X_train.shape[1]

np.random.seed(random_state)
theta = np.random.uniform(size=(n_features, n_classes))

## Huấn luyện mô hình:

In [12]:
train_accs = []
train_losses = []
val_accs = []
val_losses = []

for epoch in range(epochs):
    train_batch_losses = []
    train_batch_accs = []
    val_batch_losses = []
    val_batch_accs = []

    for i in range(0, X_train.shape[0], batch_size):
        X_i = X_train[i:i+batch_size]
        y_i = y_train[i:i+batch_size]

        y_hat = predict(X_i, theta)
        train_loss = compute_loss(y_hat, y_i)

        gradient = compute_gradient(X_i, y_i, y_hat)
        theta = update_theta(theta, gradient, lr)

        train_batch_losses.append(train_loss)

        train_acc = compute_accuracy(X_train, y_train, theta)
        train_batch_accs.append(train_acc)

        y_val_hat = predict(X_val, theta)
        val_loss = compute_loss(y_val_hat, y_val)
        val_batch_losses.append(val_loss)


        val_acc = compute_accuracy(X_val, y_val, theta)
        val_batch_accs.append(val_acc)

    train_batch_loss = sum(train_batch_losses)/len(train_batch_losses)
    val_batch_loss = sum(val_batch_losses)/len(val_batch_losses)
    train_batch_acc = sum(train_batch_accs)/len(train_batch_accs)
    val_batch_acc = sum(val_batch_accs)/len(val_batch_accs)

    train_losses.append(train_batch_loss)
    val_losses.append(val_batch_loss)
    train_accs.append(train_batch_acc)
    val_accs.append(val_batch_acc)

    print (f'\nEPOCH {epoch + 1}:\tTraining loss : {train_batch_loss:.3f} \tValidation loss : {val_batch_loss:.3f}')


EPOCH 1:	Training loss : 0.385 	Validation loss : 0.384

EPOCH 2:	Training loss : 0.385 	Validation loss : 0.384

EPOCH 3:	Training loss : 0.384 	Validation loss : 0.383

EPOCH 4:	Training loss : 0.384 	Validation loss : 0.383

EPOCH 5:	Training loss : 0.383 	Validation loss : 0.382

EPOCH 6:	Training loss : 0.383 	Validation loss : 0.382

EPOCH 7:	Training loss : 0.383 	Validation loss : 0.382

EPOCH 8:	Training loss : 0.382 	Validation loss : 0.381

EPOCH 9:	Training loss : 0.382 	Validation loss : 0.381

EPOCH 10:	Training loss : 0.382 	Validation loss : 0.381

EPOCH 11:	Training loss : 0.381 	Validation loss : 0.380

EPOCH 12:	Training loss : 0.381 	Validation loss : 0.380

EPOCH 13:	Training loss : 0.381 	Validation loss : 0.380

EPOCH 14:	Training loss : 0.380 	Validation loss : 0.379

EPOCH 15:	Training loss : 0.380 	Validation loss : 0.379

EPOCH 16:	Training loss : 0.380 	Validation loss : 0.379

EPOCH 17:	Training loss : 0.379 	Validation loss : 0.378

EPOCH 18:	Training los

In [ ]:
fig,ax = plt.subplots(2, 2, figsize=(12, 10))
ax[0, 0].plot(train_losses)
ax[0, 0].set(xlabel='Epoch', ylabel='Loss')
ax[0, 0].set_title('Training Loss')

ax[0, 1].plot(val_losses, 'orange')
ax[0, 1].set(xlabel='Epoch', ylabel='Loss')
ax[0, 1].set_title('Validation Loss')

ax[1, 0].plot(train_accs)
ax[1, 0].set(xlabel='Epoch', ylabel='Accuracy')
ax[1, 0].set_title('Training Accuracy')

ax[1, 1].plot(val_accs, 'orange')
ax[1, 1].set(xlabel='Epoch', ylabel='Accuracy')
ax[1, 1].set_title ('Validation Accuracy')

plt.show()

## Đánh giá mô hình:

In [ ]:
val_set_acc = compute_accuracy(X_val, y_val, theta)
test_set_acc = compute_accuracy(X_test, y_test, theta)
print('Evaluation on validation and test set :')
print(f'Accuracy : {val_set_acc}')
print(f'Accuracy : {test_set_acc}')